In [5]:
import cobra
import libsbml
from cobra.core import Metabolite, Reaction
import pandas as pd
from cobra import flux_analysis
ModelF = cobra.io.read_sbml_model("CAM_12PModel.xml")
ModelF.solver="glpk"
cobra.flux_analysis.pfba(ModelF)

#adding maintenance cost
PPFD = 100                            #light intensity of the model
ATPase = (0.0049*PPFD) + 2.7851       #non-growth assocaited maintenance (NGAM) cost based on light - see Topfer et al 2020 Supplemental information section 1.2.3

for i in range(1,13):
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = ATPase
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = ATPase
    
    #Setting NADPH demand to 1/3 of ATP demand and distributing this demand to cytosol, plastid and mitochondria based on Cheung et al 2013 (doi: 10.1111/tpj.12252)
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = ATPase/9
        
cobra.flux_analysis.pfba(ModelF)

,fluxes,reduced_costs
PRO_PROTON_vc1,0.000000,2.0
Ca_tx1,0.000000,2.0
H2O_xc1,0.000000,2.0
sCIT_biomass1,0.000000,2.0
ACETYLGLUTKIN_RXN_p1,0.000108,-2.0
...,...,...
THREO_DS_ISO_CITRATE_v9_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v10_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v11_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v12_accumulation,0.299252,-2.0


In [6]:
ModelF3 = ModelF.copy()

In [7]:
#Apply constraints
#Constrain ATP_ADP_Pi_pc to 0 (we know NTT, plastidic nucleotide transporter is only active in importing ATP to chloroplast at night or in non-photosynthetic tissues (Reinhold et al., 2007; Flugge et al., 2011; Voon and Lim, 2019))
for i in range(1,7):    
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).upper_bound = 0
    
#Constrain PEPCK to 0 (because we are modelling Phalaenopsis) (ME takes over as decarboxylating enzyme)    
for i in range(1,13):    
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).upper_bound = 0

#Remove reaction H_pc (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_pc"+str(i)).upper_bound = 0

#Constrain a flux ratio between NAD-ME and NADP-ME according to temporal enzyme activity measurements in Kalanchoë and Dever et al. 2015
for i in range(1,13):
    new_constraint1 = ModelF3.problem.Constraint(
        8*(ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i)).flux_expression + ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i)).flux_expression) - ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint1)
    
#Constrain a flux ratio between PPDK_c and PPDK_p according to Kondo et al. 2000 and Dever et al. 2015
for i in range(1,13):
    new_constraint2 = ModelF3.problem.Constraint(
        2*ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint2)
    
#Change direction of mitochondrial PYR-H+ symporter (PiC) allowing only PYR import into mitonchondria (Le et al., 2021)
for i in range(1,13):    
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).upper_bound = 0
    
#Add a PYR channel reaction to re-allow PYR export from mitochondria
for i in range(1,13):
    rxn = Reaction("PYR_mc"+str(i)+"_channel",name = "PYR_mc"+str(i)+"_channel")
   
    rxn.add_metabolites({ModelF3.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
                         ModelF3.metabolites.get_by_id("PYRUVATE_c"+str(i)):1})
   
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    ModelF3.add_reactions([rxn,])
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)

In [223]:
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
cobra.io.write_sbml_model(ModelF3, "CAM_12P_ME_Model.xml")

In [201]:
from Functions import checkProtonFluxes
checkProtonFluxes(ModelF3, tag="ME")

In [202]:
#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCactive.csv","w")

for rxn in ModelF3.reactions:
    fout.write(rxn.id+","+rxn.name+","+rxn.reaction+","+str(rxn.flux)+"\n")
    
fout.close()

In [208]:
#Additional checks that are reported in the manuscript

ModelF4 = ModelF3.copy()

# Check importance of the PiC transporter 
for i in range(1,13):
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).upper_bound = 0
    
#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0

sol4 = flux_analysis.parsimonious.pfba(ModelF4)

#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCoff.csv","w")

for rxn in ModelF4.reactions:
    fout.write(rxn.id+","+rxn.name+","+rxn.reaction+","+str(rxn.flux)+"\n")
    
fout.close()

In [209]:
#Simulate different mitochondrial pyruvate export mechanisms (symport and antiport)
#Uncomment one to simulate, leave the other commented

#Reactivate PiC if set to 0 in previous code block
for i in range(1,13):
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).lower_bound = -1000
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).upper_bound = 1000
    
#PYR-H symport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_symport",name = "PYR_H_mc"+str(i)+"_symport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)
#     #turn off PYR channel
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").upper_bound = 0
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").lower_bound = 0

# PYR-H antiport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_antiport",name = "PYR_H_mc"+str(i)+"_antiport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):-1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)
#     #turn off PYR channel
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").upper_bound = 0
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").lower_bound = 0

#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0
    
sol4 = flux_analysis.parsimonious.pfba(ModelF4)

In [192]:
#Summary of fluxes through particular CAM reactions when checking different mitochondrial pyruvate export mechanisms
starch = 0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("GLYCOGENSYN_RXN_p"+str(i))
    #print(rxn.reaction)
    #print(rxn.flux)
    starch = starch+rxn.flux
print("---------------")
print("Total starch synthesized: "+str(starch))

MPC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i))
    #print(rxn.reaction)
    #print(rxn.flux)
    MPC=MPC+rxn.flux
print("---------------")
print("Total flux through MPC: "+str(MPC))

#uncomment one of the mechanisms to check flux
# PYRsymport=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_symport")
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRsymport=PYRsymport+rxn.flux
# print("---------------")
# print("Total flux through PYRsymport: "+str(PYRsymport))

# PYRantiport=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_antiport")
#     #print(rxn.reaction)
#     #print(rxn.flux)
#     PYRantiport=PYRantiport+rxn.flux
# print("---------------")
# print("Total flux through PYRantiport: "+str(PYRantiport))

# PYRchannel=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel")
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRchannel=PYRchannel+rxn.flux
# print("---------------")
# print("Total flux through PYRchannel: "+str(PYRchannel))

PEPC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PEPCARBOX_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PEPC=PEPC+rxn.flux
print("---------------")
print("Total flux through PEPC: "+str(PEPC))

NADME=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADME=NADME+rxn.flux
print("---------------")
print("Total flux through NADME: "+str(NADME))

NADPMEcyt=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEcyt=NADPMEcyt+rxn.flux
print("---------------")
print("Total flux through NADPMEcyt: "+str(NADPMEcyt))

NADPMEplast=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEplast=NADPMEplast+rxn.flux
print("---------------")
print("Total flux through NADPMEplast: "+str(NADPMEplast))

PPDKcyt=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKcyt=PPDKcyt+rxn.flux
print("---------------")
print("Total flux through PPDKcyt: "+str(PPDKcyt))

PPDKplast=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKplast=PPDKplast+rxn.flux
print("---------------")
print("Total flux through PPDKplast: "+str(PPDKplast))

RubOxy=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("RXN_961_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    RubOxy=RubOxy+rxn.flux
print("---------------")
print("Total flux through RubOxy: "+str(RubOxy))

RubCarboxy=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    RubCarboxy=RubCarboxy+rxn.flux
print("---------------")
print("Total flux through RubCarboxy: "+str(RubCarboxy))

MalateEfflux=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_rev_vc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    MalateEfflux=MalateEfflux+rxn.flux
print("---------------")
print("Total flux through MalateEfflux: "+str(MalateEfflux))

PiC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PiC=PiC+rxn.flux
print("---------------")
print("Total flux through PiC: "+str(PiC))

GAPDH=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    GAPDH=GAPDH+rxn.flux
print("---------------")
print("Total flux through GAPDH: "+str(GAPDH))

MalImp=0
for i in range (1,13):  
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_vc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    MalImp=MalImp+rxn.flux
print("---------------")
print("Total malate imported in vacuole: "+str(MalImp))

ATPdayMito = 0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("Mitochondrial_ATP_Synthase_m"+str(i))
    ATPdayMito += rxn.flux
print("---------------")
print("Total flux through ATP synthase in mitochondria:" +str(ATPdayMito))

phloem_rxn = ModelF4.reactions.get_by_id("Diel_phloem_export")
print("---------------")
print("Diel phloem export flux: " +str(phloem_rxn.flux))

---------------
Total starch synthesized: 14.306682068797429
---------------
Total flux through MPC: -3.074460318119163
---------------
Total flux through PYRantiport: 22.199878868233615
---------------
Total flux through PEPC: 25.092548413598696
---------------
Total flux through NADME: 22.199878868233633
---------------
Total flux through NADPMEcyt: 2.774984858529204
---------------
Total flux through NADPMEplast: 0.0
---------------
Total flux through PPDKcyt: 16.6314662412904
---------------
Total flux through PPDKplast: 8.3157331206452
---------------
Total flux through RubOxy: 0.0
---------------
Total flux through RubCarboxy: 31.028534706138146
---------------
Total flux through MalateEfflux: 23.637099662293373
---------------
Total flux through PiC: -133.78751920498178
---------------
Total flux through GAPDH: 2.701431512860773
---------------
Total malate imported in vacuole: 22.47436477085381
---------------
Total flux through ATP synthase in mitochondria:36.52162832482762
--